# 静态方法和类方法

## 特性（property） 

### 什么是特性（property）

- property是一种特殊的属性，访问它时会执行一段功能（函数）然后返回值

In [13]:
import math
class Circle:
    def __init__(self,radius):  # 圆的半径radius
        self.radius=radius

    @property
    def area(self):
        return math.pi * self.radius**2  # 计算面积

    @property
    def perimeter(self):
        return 2*math.pi*self.radius # 计算周长

c=Circle(10)
print(c.radius)
print(c.area)  # 可以向访问数据属性一样去访问area,会触发一个函数的执行,动态计算出一个值
print(c.perimeter)  # 同上

10
314.1592653589793
62.83185307179586


- 需要注意的是每次访问都是动态的，多次访问会多次执行函数。

In [14]:
c.area = 3  # 为特性area赋值

AttributeError: can't set attribute

- 注意：此时的特性arear和perimeter不能被赋值

In [85]:
import math
class Circle:
    def __init__(self,radius):  # 圆的半径radius
        self.radius=radius

    def area(self):
        return math.pi * self.radius**2  # 计算面积

    @property
    def perimeter(self):
        return 2*math.pi*self.radius # 计算周长

c=Circle(10)
print(c.radius)
print(c.area)  # 可以向访问数据属性一样去访问area,会触发一个函数的执行,动态计算出一个值
print(c.area())
print(c.perimeter)  # 同上

10
<bound method Circle.area of <__main__.Circle object at 0x0000021833503070>>
314.1592653589793
62.83185307179586


- 这是一个对比，如果没有使用`特性`，则需要以函数形式访问，否则只会输出函数对象所在的地址。

In [17]:
c.area = 3
print(c.area)

3


- 非特性则可以赋值。

### 为什么要用property

将一个类的函数定义成特性以后，对象再去使用的时候`obj.name`,根本无法察觉自己的name是执行了一个函数然后计算出来的，这种特性的使用方式遵循了统一访问的原则。

    ps：面向对象的封装有三种方式:
    【public】
    这种其实就是不封装,是对外公开的
    【protected】
    这种封装方式对外不公开,但对朋友(friend)或者子类(形象的说法是“儿子”,但我不知道为什么大家 不说“女儿”,就像“parent”本来是“父母”的意思,但中文都是叫“父类”)公开
    【private】
    这种封装对谁都不公开
    
python并没有在语法上把它们三个内建到自己的class机制中，在C++里一般会将所有的数据都设置为私有的，然后提供set和get方法（接口）去设置和获取，在python中通过property方法可以实现

In [28]:
class Foo:
    def __init__(self,val):
        self.__NAME=val  # 将所有的数据属性都隐藏起来

    @property
    def name(self):
        return self.__NAME  # obj.name访问的是self.__NAME(这也是真实值的存放位置)

    @name.setter
    def name(self,value):
        if not isinstance(value,str):  # 在设定值之前进行类型检查
            raise TypeError('%s must be str' % value)
        self.__NAME=value #通过类型检查后,将值value存放到真实的位置self.__NAME

    @name.deleter
    def name(self):
        raise TypeError('Can not delete')

f=Foo('Jx')
print(f.name)

Jx


In [32]:
# f.name = 10  # 抛出异常'TypeError: 10 must be str'
# del f.name  # 抛出异常'TypeError: Can not delete'

- 通过这样的方式，可以设定条件更改 特性。

---

通常情况下，在类中定义的所有函数（注意了，这里说的就是所有，跟self啥的没关系，self也只是一个再普通不过的参数而已）都是对象的绑定方法，对象在调用绑定方法时会自动将自己作为参数传递给方法的第一个参数。除此之外还有两种常见的方法：静态方法和类方法，二者是为类量身定制的，但是实例非要使用，也不会报错。

## 静态方法（staticmethod）

python staticmethod 返回函数的静态方法。

该方法不强制要求传递参数。

In [33]:
class Foo:
    def spam(x, y, z):  # 类中的一个函数，千万不要懵逼，self和x啥的没有不同都是参数名
        print(x, y, z)
    spam = staticmethod(spam)  # 把spam函数做成静态方法

`@staticmethod` 等同于 `spam=staticmethod(spam)`,于是可以写成

In [54]:
class Foo:
    @staticmethod  # 装饰器
    def spam(x, y, z):
        print(x, y, z)


print(type(Foo()))
print(type(Foo.spam))  # 类型本质就是函数
Foo.spam(1, 2, 3)  # 调用函数应该有几个参数就传几个参数

Foo().spam(3, 3, 3)  # 实例也

<class '__main__.Foo'>
<class 'function'>
1 2 3
3 3 3


使用静态方法后就不会传入self参数。
同时也说明通过静态方法 `spam`，从而可以实现实例化使用`Foo().spam`，当然也可以不实例化调用该方法 `Foo.spam()`。

- 对比一下

In [50]:
class Foo:
#     @staticmethod  # 装饰器
    def spam(x, y, z):
        print(x, y, z)

In [51]:
print(type(Foo()))
print(type(Foo.spam))  # 类型本质就是函数
Foo.spam(1, 2, 3)  # 调用函数应该有几个参数就传几个参数

Foo().spam(3, 3, 3)  # 没有静态方法，则会自动传入一个self参数。

<class '__main__.Foo'>
<class 'function'>
1 2 3


TypeError: spam() takes 3 positional arguments but 4 were given

- 应用场景:编写类时需要采用很多不同的方式来创建实例，而我们只有一个`__init__`函数，此时静态方法就派上用场了。

In [66]:
import time


class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @staticmethod
    def now():  # 用Date.now()的形式去产生实例,该实例用的是当前时间
        t = time.localtime()  # 获取结构化的时间格式
        return Date(t.tm_year, t.tm_mon, t.tm_mday)  # 新建实例并且返回

    @staticmethod
    def tomorrow():  # 用Date.tomorrow()的形式去产生实例,该实例用的是明天的时间
        t = time.localtime(time.time()+86400)
        return Date(t.tm_year, t.tm_mon, t.tm_mday)


a = Date('1987', 11, 27)  # 自己定义时间
b = Date.now()  # 采用当前时间
c = Date.tomorrow()  # 采用明天的时间

print(a.year, a.month, a.day)
print(b.year, b.month, b.day)
print(c.year, c.month, c.day)

print(type(Date.now),type(Date.now()))

1987 11 27
2021 7 15
2021 7 16
<class 'function'> <class '__main__.Date'>


## 类方法（classmethod） 

- 类方法是给类用的，类在使用时会将类本身当做参数传给类方法的第一个参数，python为我们内置了函数classmethod来把类中的函数定义成类方法

In [67]:
class A:
    x = 1

    @classmethod
    def test(cls):
        print(cls, cls.x)


class B(A):
    x = 2


B.test()

<class '__main__.B'> 2


- 应用场景

In [71]:
import time


class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @staticmethod
    def now():
        t = time.localtime()
        return Date(t.tm_year, t.tm_mon, t.tm_mday)


class EuroDate(Date):
    def __str__(self):
        return 'year:%s month:%s day:%s' % (self.year, self.month, self.day)


e = EuroDate.now()
print(e)  

我们的意图是想触发`EuroDate.__str__`,但是结果却不是这样。

因为e就是用Date类产生的,所以根本不会触发`EuroDate.__str__`,解决方法就是用`classmethod`

In [73]:
import time


class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    # @staticmethod
    # def now():
    #     t=time.localtime()
    #     return Date(t.tm_year,t.tm_mon,t.tm_mday)

    @classmethod  # 改成类方法
    def now(cls):
        t = time.localtime()
        return cls(t.tm_year, t.tm_mon, t.tm_mday)  # 哪个类来调用,即用哪个类cls来实例化


class EuroDate(Date):
    def __str__(self):
        return 'year:%s month:%s day:%s' % (self.year, self.month, self.day)


e = EuroDate.now()
print(e)  # 我们的意图是想触发EuroDate.__str__,此时e就是由EuroDate产生的,所以会如我们所愿

year:2021 month:7 day:15


强调，注意注意注意：静态方法和类方法虽然是给类准备的，但是如果实例去用，也是可以用的，只不过实例去调用的时候容易让人混淆，不知道你要干啥

In [74]:
x=e.now() #通过实例e去调用类方法也一样可以使用,静态方法也一样
print(x)

year:2021 month:7 day:15


## 附加知识点__str__的用法            

`__str__`定义在类内部，必须返回一个字符串类型，

什么时候会出发它的执行呢？打印由这个类产生的对象时，会触发执行

In [77]:
class People:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        return '<name:%s,age:%s>' % (self.name, self.age)


p1 = People('Jx', 18)
print(p1)

<name:Jx,age:18>


# 一个案例加深理解

In [82]:
class Dog:
    # 类变量：每个类的类变量是相同的
    # 所有的狗类都会有同一个dogbook，因此dogbook不是一个实例，并且属于狗类。
    dogbook = {"黄色": 30, "黑色": 20, "白色": 0}

    # 实例变量：每个实例的实例变量可以不同
    def __init__(self, name, color, weight):  # 每个狗都有不一样的属性，则这些是狗的实例。
        self.name = name
        self.color = color
        self.weight = weight
        # 此处省略若干行，应该更新dogbook的数量

    # 实例方法: 定义时,必须把self作为第一个参数，可以访问实例变量，只能通过实例名访问
    # 当然，调用的时候python会自动帮忙上传self参数，但是，这就意味着，实例方法是必须通过实例名访问
    # 如果不实例调用：Dog.bark()  报错：# TypeError: bark() missing 1 required positional argument: 'self'
    # Dog.bark 只会打印 function 的名字：# <function __main__.Dog.bark(self)>
    def bark(self):
        print(f"{self.name} 叫了起来")

    # 类方法：定义时,必须把类作为第一个参数，可以访问类变量，可以通过实例名或类名访问
    # 也就是说使用 Dog.dog_num() 和 Dog().dog_num() 结果是一样的。
    @classmethod
    def dog_num(cls):
        num = 0
        for v in cls.dogbook.values():
            num = num + v
        return num

    # 静态方法：不强制传入self或者cls, 他对类和实例都一无所知。不能访问类变量，也不能访问实例变量；可以通过实例名或类名访问。
    # 静态方法可以写在类外面也可以写在类里面，写在类里面的原因是与这个类息息相关的。
    @staticmethod
    def total_weights(dogs):
        total = 0
        for o in dogs:
            total = total + o.weight
        return total


print(f"共有 {Dog.dog_num()} 条狗")
d1 = Dog("大黄", "黄色", 10)
d1.bark()
print(f"共有 {d1.dog_num()} 条狗")

d2 = Dog("旺财", "黑色", 8)
d2.bark()

print(f"狗共重 {Dog.total_weights([d1, d2])} 公斤")

共有 50 条狗
大黄 叫了起来
共有 50 条狗
旺财 叫了起来
狗共重 18 公斤
